In [10]:
# !pip install pypdf cohere tiktoken langchain sentence_transformers openai pinecone-client -q
# !pip install -U langchain-google-genai pillow

# import streamlit as st
import os
from pathlib import Path
# import numpy as np
# import requests
import pinecone
from pprint import pprint

from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
import google.generativeai as genai

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

os.chdir('C:/Users/Gwool/Documents/Python Scripts/LLM')

Functions

In [11]:
def create_chunks(doc_to_chunk):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=100,
        length_function=len
        )
    return text_splitter.split_documents(doc_to_chunk)

# -----------------------------------------------------

def load_pdf(path):
    loader = PyPDFLoader(path)
    return loader.load()

# -----------------------------------------------------

def load_chunck_file(path):
    doc = load_pdf(path)
    return create_chunks(doc)


Create Pinecone Index

In [5]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENV = "gcp-starter"
index_name = 'test-llm'
dim = 768

pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)

pinecone.create_index(index_name, dimension = dim, metric = "cosine")

print(pinecone.describe_index(index_name))

index = pinecone.Index(index_name)

Load PDF File

In [ ]:
file_path = './data/DNU-Milei-2023.pdf'

chunks = load_chunck_file(file_path)
print('N° Chunks:', len(chunks))

N° Chunks: 371


Embedings

In [14]:
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name = model_name)

Load chunck PDF File into Pinecone Database

In [ ]:
Pinecone.from_documents(chunks, embeddings, index_name=index_name)

Conversation with Gemini-Pro - Give context

In [15]:
from langchain import PromptTemplate

prompt_template = """
Por favor, responde a la pregunta con el mayor detalle posible en base al contexto proporcionado.
Asegúrate de incluir todos los detalles relevantes. Si la respuesta no está disponible en el contexto proporcionado, por favor responde con 
"La respuesta no está disponible en el contexto". Evita proporcionar respuestas incorrectas.
\n\n
  Context:\n {context}?\n
  Question: \n{question}\n

  Answer:
"""

prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])

In [16]:
vstore = Pinecone.from_existing_index(index_name, embeddings)
llm = ChatGoogleGenerativeAI(model = "gemini-pro", convert_system_message_to_human = True, temperature = 0.3)
chain = load_qa_chain(llm, chain_type = "stuff", prompt = prompt) # map_reduce - stuff

In [20]:
def data_querying(question):
    vstore = Pinecone.from_existing_index(index_name, embeddings)
    llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True, temperature=0.3)
    chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt)  # map_reduce - stuff

    docs = vstore.similarity_search(question, 3)

    response = chain({"input_documents":docs, "question": question}, return_only_outputs=True)
    response_text = response.get('output_text')

    return pprint(response_text)

In [21]:
data_querying('¿Que medidas se toman sobre las empresas del estado? Detalla la respuesta')

('- Se eliminan las figuras jurídicas de las Sociedades del Estado, reguladas '
 'por la Ley N° 20.705, las Empresas del Estado previstas en la Ley N° 13.653 '
 'y las Sociedades de Economía Mixta contempladas en el Decreto - Ley N° '
 '15.349/46.\n'
 '- Se modifica el capítulo del Programa de Propiedad Participada de la Ley N° '
 '23.696, a los fines de facilitar el traspaso de las acciones de las empresas '
 'actualmente estatales a sus empleados.\n'
 '- Se incluyen cambios en la Ley N° 21.799 para adecuar al BANCO DE LA NACIÓN '
 'ARGENTINA a su nueva configuración societaria.\n'
 '- Se modifica el status jurídico de las empresas públicas, reconvirtiéndolas '
 'en Sociedades Anónimas, acordes al régimen de la Ley General de Sociedades.')
